In [1]:
import json
import pandas as pd
import glob

In [2]:
test_json_dir="database/1103output/test.json"

with open(test_json_dir,"r") as f:
    test_list=json.load(f)

In [20]:
test_ref_ids=[i["ref_id"] for i in test_list if "ref_id" in i]
len(test_ref_ids)

250

In [3]:
context_dirs=glob.glob("database/1103output/context_list*.json")

context_list_list=[]
for path in sorted(context_dirs):
    with open(path,"r") as f:
        l=json.load(f)
    context_list_list.append(l)

In [8]:
#introduction target

context_ids=[i["context_id"] for i in test_list if "context_id" in i]
context_ids=list(set(context_ids))

extracted_context_sets=[]

for context_list in context_list_list:
    extracted_items = [context_list[i] for i in context_ids if i < len(context_list)]
    extracted_context_sets.extend(extracted_items)

with open("database/1111split/target/intro_eng.json","w") as f:
    json.dump(extracted_context_sets[:len(context_ids)],f,indent=4)

with open("database/1111split/target/intro_esp_ger_ita.json","w") as f:
    json.dump(extracted_context_sets[len(context_ids):],f,indent=4)

In [17]:
len(concn_list)

13789

In [9]:
# irrelevant 1 / intro
import random
extracted_context_sets=[]
for context_list in context_list_list:
    excluded_items = [item for idx, item in enumerate(context_list) if idx not in context_ids]
    extracted_context_sets.extend(excluded_items)

with open(f"database//1111split/irrelevant1/intro_eng.json","w") as f:
    json.dump(extracted_context_sets[:len(excluded_items)],f,indent=4)
with open(f"database//1111split/irrelevant1/intro_esp_ger_ita.json","w") as f:
    json.dump(extracted_context_sets[len(excluded_items):],f,indent=4)

# abst concn

In [28]:
import re
def clean_text(text):
    pattern = r'\.(?=[A-Za-z])'
    text= re.sub(pattern, '. ', text)
    #連続したピリオドを削除
    text=re.sub(r'\.{2,}', '.', text)
    return text

def unify_text(text):
    text = text.replace("。", "\n")
    text = text.replace(".", "\n")
    text = text.replace("．", "\n")
    text = text.replace("？", "\n")
    text = text.replace("?", "\n")
    text = text.replace("！", "\n")
    text = text.replace("!", "\n")
    text = text.replace("；", "\n")
    text = text.replace(";", "\n")
    text = text.replace("：", "\n")
    text = text.replace(":", "\n")
    text = text.replace("　", " ")

    return text

def split_text(text, chunk_size_limit=2000):
    text = unify_text(text)

    text_list = text.split("\n")

    chunk_text_list = []
    temp_text = ""
    for t in text_list:
        t = t.strip()
        temp_text += t+"."
        if len(temp_text) < chunk_size_limit:
            continue
        else:
            chunk_text_list.append(temp_text)
            temp_text = ""

    chunk_text_list.append(clean_text(temp_text))

    chunk_text_list = [t for t in chunk_text_list if len(t) > 10]
    return chunk_text_list

In [13]:
#abst
with open("database/1103output/ref_list.json","r") as f:
    ref_list=json.load(f)

with open("database/1103output/context_to_ref_id.json","r") as f:
    context_to_ref_id=json.load(f)

In [15]:

used_ref_list=list(set([r["ref_id"] for r in context_to_ref_id]))
abst_list=[r["abstract"] for r in ref_list if r["ref_id"] in used_ref_list]
concn_list=[r["main"][2]["text"] for r in ref_list if r["ref_id"] in used_ref_list if len(r["main"])>3]

In [36]:
from tqdm import tqdm
target_chunk_abst_list=[]
irrelevant_chunk_abst_list=[]
target_chunk_concn_list=[]
irrelevant_chunk_concn_list=[]


for r in tqdm(ref_list):
    abst=r["abstract"]

    try:
        concn=r["main"][2]["text"]
    except:
        concn=""

    chunk_abst=[]
    chunk_concn=split_text(concn)
    chunk_abst=split_text(abst)

    if r["ref_id"] in test_ref_ids:
        target_chunk_abst_list.extend(chunk_abst)
        target_chunk_concn_list.extend(chunk_concn)
    elif r["ref_id"] in used_ref_list:
        irrelevant_chunk_abst_list.extend(chunk_abst)
        irrelevant_chunk_concn_list.extend(chunk_concn)


100%|██████████| 64666/64666 [00:22<00:00, 2816.03it/s]


In [37]:
len(target_chunk_abst_list),len(target_chunk_concn_list),len(irrelevant_chunk_abst_list),len(irrelevant_chunk_concn_list)

(250, 950, 13829, 46222)

In [38]:
with open("database/1111split/target/abst_eng.json","w") as f:
    json.dump(target_chunk_abst_list,f,indent=4)

with open("database/1111split/target/concn_eng.json","w") as f:
    json.dump(target_chunk_concn_list,f,indent=4)

with open("database/1111split/irrelevant1/abst_eng.json","w") as f:
    json.dump(irrelevant_chunk_abst_list,f,indent=4)

with open("database/1111split/irrelevant1/concn_eng.json","w") as f:
    json.dump(irrelevant_chunk_concn_list,f,indent=4)


In [41]:
#instruct
!cp database/1103output/train.json database/1111split/irrelevant1/instruct_eng.json
!cp database/1103output/test.json database/1111split/target/instruct_eng.json

In [53]:
# irrelevant 2 intro

irreleavnt2_intro_list=[]
for r in tqdm(ref_list):
    if r["ref_id"] not in used_ref_list:
        intro=r["main"][0]["text"]
        intro_chunk=split_text(intro)
        irreleavnt2_intro_list.extend(intro_chunk)

100%|██████████| 64666/64666 [00:17<00:00, 3759.43it/s]


In [ ]:
with open("database/1111split/irrelevant2/intro_eng.json","w") as f:
    json.dump(irreleavnt2_intro_list,f,indent=4)

In [55]:
!cp database/1103output/mmlu_train.json database/1111split/mmlu_train.json

In [1]:
import json

with open("database/1111split/irrelevant2/intro_eng.json","r") as f:
    irreleavnt2_intro_list=json.load(f)
len(irreleavnt2_intro_list)

136092